In [25]:
import pandas as pd
import numpy as np

### Import Billing Data

In [26]:
# Import Billing File
file_path = (
    r"C:\Users\pirat\Dropbox\Consulting Inc\Upperline Health\1. Payor Data & Contracts"
    r"\99-Athena Reports\ULH All Payors FL - DOS 2024 & YTD June 2025 - as of 11.17.25.xlsx"
)

# Read all sheets (dict: {sheet_name: DataFrame})
sheets_dict = pd.read_excel(file_path, sheet_name=None)

# If the file really only has those 2 relevant sheets, this will grab both
df_raw = pd.concat(sheets_dict.values(), ignore_index=True)

# delete duplicate rows
df_raw = df_raw.drop_duplicates(keep='first')

# BENCHMARK: 1m per 1 mil rows

In [27]:
# Initial cleaning
df_clean = df_raw.copy()

# change net payment from negative to positive
df_clean['Net Payment'] = df_clean['Net Payment'].abs()

# create avg paid per claim
df_clean['Avg_Paid_Per_Claim'] = (df_clean['Net Payment']/df_clean['Charge Units']).fillna(0)

# create key from Claim ID and Patient ID
df_clean["Patient_Claim_Key"] = (
    df_clean["Claim ID"].astype(str)
    + "-"
    + df_clean["Patient ID"].astype(str)
    + "-"
    + df_clean["Date of Service"].astype(str)
)

# Get "State" from first two characters of ["Service Department"]
df_clean["State"] = df_clean["Service Department"].str[:2]

### Import Payor Map

In [28]:
# Import Payer Crosswalk File
file_path = (
    r"C:\Users\pirat\Dropbox\Consulting Inc\Upperline Health\1. Payor Data & Contracts\99-Athena Reports"
    r"\Payor Mapping_FIN.xlsx"
)

# Read only "Payor Mapping_FIN" sheet
df_payor_mapping = pd.read_excel(file_path, sheet_name="PayorMap_FIN")

df_payor_mapping["INSNAME_OPS_key"] = (
    df_payor_mapping["INSNAME_OPS"].str.upper().str.strip()
)

# drop unused columns
df_payor_mapping = df_payor_mapping[[
    "INSNAME_OPS_key",
    "INSNAME_OPS",
    "PAYORCAT_FIN",
    "PARENTCO_NAME_FIN"
]]

# delete duplicate rows
df_payor_mapping = df_payor_mapping.drop_duplicates("INSNAME_OPS_key", keep='first')

### Merge Payor Map to Billing Data

In [29]:
# Normalize keys to uppercase, stripped
df_clean["Primary Insurance_key"] = (
    df_clean["Primary Insurance"].str.upper().str.strip()
)

# Merge on the normalized keys
df_merged = pd.merge(
    df_clean,
    df_payor_mapping[["INSNAME_OPS_key", "INSNAME_OPS", "PAYORCAT_FIN", "PARENTCO_NAME_FIN"]],
    how="left",
    left_on="Primary Insurance_key",
    right_on="INSNAME_OPS_key",
)

# drop helper columns
df_merged = df_merged.drop(columns=["Primary Insurance_key", "INSNAME_OPS_key"])

### Additional Cleaning

In [30]:
# Filling in PAYORCAT_FIN. 
# First make a series based on the raw insurance name
ins = df_merged["INSNAME_OPS"].fillna("").str.lower()

# build a guess for every row
payor_guess = np.select(
    [
        ins.str.contains("medicare") | ins.str.contains("mcr"),
        ins.str.contains("medicaid") | ins.str.contains("mcd"),
        ins.str.contains("tricare") | ins.str.contains("exchange"),
    ],
    ["MEDICARE", "MEDICAID", "OTHER"],
    default="COMMERCIAL",
)

# use it only where PAYORCAT_FIN is null
df_merged["PAYORCAT_FIN"] = df_merged["PAYORCAT_FIN"].fillna(
    pd.Series(payor_guess, index=df_merged.index)
)

In [31]:
# delete any rows where "Date of Service" is before 2024-01-01
df_merged = df_merged[
    pd.to_datetime(df_merged["Date of Service"]) >= pd.to_datetime("2024-01-01")
]

In [32]:
import pandas as pd

parts = (
    df_merged["Procedure Code"]
      .fillna("")
      .astype(str)
      .str.split(",", expand=True)
      .apply(lambda s: s.str.strip())
      .replace("", pd.NA)
)

# rename: first col = CPT_Code_Core, remaining = Mod1, Mod2, ...
parts.columns = ["CPT_Code_Core"] + [f"Mod{i}" for i in range(1, parts.shape[1])]

df_merged = df_merged.join(parts)


In [33]:
# Set of price-altering modifiers
PRICE_ALTERING_MODS = {
    "26","TC",
    # "50","51",
    # "52","53",
    # "54","55","56",
    # "62","66",
    # "80","81","82","AS",
    # "LT","RT",  # uncomment if you want laterality treated as price-altering
}

mod_cols = [c for c in df_merged.columns if c.startswith("Mod")]

# Normalize modifier cells (strip, uppercase, handle blanks)
mods = (
    df_merged[mod_cols]
      .astype("string")
      .apply(lambda s: s.str.strip().str.upper())
      .replace({"": pd.NA, "NAN": pd.NA, "NONE": pd.NA})
)

# First matching price-altering modifier, left-to-right (Mod1, Mod2, ...)
df_merged["Price_Modifier"] = (
    mods.where(mods.isin(PRICE_ALTERING_MODS))
        .bfill(axis=1)
        .iloc[:, 0]
)

df_merged["Has_Price_Modifier"] = df_merged["Price_Modifier"].notna()

# Build Code_Final = CPT_Code_Core + "-" + first price-altering modifier (if any)
core = df_merged["CPT_Code_Core"].astype("string").str.strip()

df_merged["Code_Final"] = core.where(
    df_merged["Price_Modifier"].isna(),
    core + "-" + df_merged["Price_Modifier"],
)


### Import Service Category Map

In [39]:
# import service category mapping
file_path = (
    r"C:\Users\pirat\Dropbox\Consulting Inc\Upperline Health\1. Payor Data & Contracts\Service Code List - Upperline.xlsx"
)

df_svc_cat = pd.read_excel(file_path)

In [40]:
# Merge service category mapping
df_tested = df_merged.copy()
df_tested = pd.merge(
    df_tested,
    df_svc_cat[["Code Full", "Category Name", "Sub-Category Name"]],
    how="left",
    left_on="Code_Final",
    right_on="Code Full")

In [ ]:
# export to Excel
output_path = (
    r"C:\Users\pirat\Dropbox\Consulting Inc\Upperline Health\1. Payor Data & Contracts\99-Athena Reports"
    r"\ULH_Billing_Processed.xlsx"
)

df_tested.to_excel(output_path, index=False)